In [ ]:
#install python library
pip install tkintertable
pip install pandastable
pip install folium

In [8]:
#import python library
import numpy as np
import pandas as pd
import re
import os.path
import seaborn as sns
import matplotlib.pyplot as plt
import json
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox as msg
from pandastable import Table, TableModel
from tkintertable import TableCanvas
import tkinter.ttk
import time
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import folium
import json
from collections import OrderedDict
%matplotlib inline

## Data Analysis Program

In [10]:
class application :
    def __init__(self, root) :
        #Program status JSON File
        self.file = './data/status.json'
        
        #Store and load program status
        if os.path.isfile(self.file) :
            #Load program status from JSON File
            with open(self.file, 'r') as json_file :
                self.file_data = json.load(json_file)
                self.status = self.file_data['status']
                self.file_name = self.file_data['file_name']
                if os.path.isfile(self.file_name) :
                    self.final_data = pd.read_json(self.file_name)
                else :
                    self.final_data = pd.DataFrame()
        else :
            #Store program status from JSON File
            self.file_data = OrderedDict()
            self.status = 'INACTIVE'
            self.file_name = ''
            self.file_data['status'] = self.status
            self.file_data['file_name'] = self.file_name
            self.final_data = pd.DataFrame()
            with open(self.file, 'w', encoding='utf-8') as make_file :
                json.dump(self.file_data, make_file, ensure_ascii=False, indent='\t')
                            
        self.root = root
        
        #Craete Menubar GUI
        menubar = Menu(root)
        filemenu = Menu(menubar)
        filemenu.add_command(label='Save JSON', font = ('Arial', 12), command=self.onSave)
        filemenu.add_command(label='Open JSON', font = ('Arial', 12), command=self.onOpen)
        filemenu.add_command(label='Exit', font = ('Arial', 12), command=self.root.quit)
        filemenu2 = Menu(menubar)
        filemenu2.add_command(label='Vendor Seating', font = ('Arial', 12), command=self.onSeat)
        filemenu2.add_command(label='Zip Code', font = ('Arial', 12), command=self.onZipcode)   
        filemenu3 = Menu(menubar)
        filemenu3.add_command(label='Number of Violation', font = ('Arial', 12), command=self.onNum)
        filemenu3.add_command(label='Save Violation Map', font = ('Arial', 12), command=self.onMap)
        
        #Craete Sub menubar GUI
        menubar.add_cascade(label='File', menu=filemenu, font = ('Arial', 12))
        menubar.add_cascade(label='Inspection Score', menu=filemenu2, font = ('Arial', 12))
        menubar.add_cascade(label='Visualization', menu=filemenu3, font = ('Arial', 12))
        menubar.add_cascade(label='Clear Data', font = ('Arial', 12), command=self.onClear)
        root.config(menu=menubar)
        
        #Craete Main Frame
        self.f = Frame(self.root, height=150, width=800)
        self.f.pack()
        
        #Craete Button and Label GUI
        self.generate_button = Button(self.f, text='1)Import Data Files',font = ('Arial', 13),  command=self.generate_file)
        self.clean_button = Button(self.f, text='2)Preparing Data', font = ('Arial', 13), command=self.prepared_file)
        self.show_button = Button(self.f, text='3)Show Prepared Data', font = ('Arial', 13), command=self.show_data)
        self.status_label = Label(self.f, text='- PROGRAM STATUS : ', font = ('Arial', 12))
        self.status_label2 = Label(self.f, text=self.status, font = ('Arial', 12), bg = 'Green')
        self.data_label = Label(self.f, text='- The prepared data set ', font = ('Arial', 12))

        #Place the widget on the frame
        self.generate_button.place(x=140, y=30)
        self.clean_button.place(x=300, y=30)
        self.show_button.place(x=450, y=30)
        self.status_label.place(x=270, y=80)
        self.status_label2.place(x=450, y=80)
        self.data_label.place(x=10, y=120)
    
    #Clear Database
    def onClear(self) :
        if os.path.isfile(self.file) :
            self.final_data = pd.DataFrame()
            self.file_data['status'] = 'INACTIVE'
            self.file_data['file_name'] = ''
            self.status_label2.configure(text=self.file_data['status'], font = ('Arial', 13), bg = 'Red')
            with open(self.file, 'w', encoding='utf-8') as make_file :
                json.dump(self.file_data, make_file, ensure_ascii=False, indent='\t')
            if os.path.isfile('./data/import_file.json') :
                os.remove('./data/import_file.json')
            if os.path.isfile('./data/prepared_file.json') :
                os.remove('./data/prepared_file.json')
            
            msg.showinfo('Clear Database', 'Clear Database')
               
    
    #import the data file(Inspections.csv, violations.csv)
    def generate_file(self) :
        if(len(self.final_data) != 0) :
            msg.showinfo('Status is Active', 'Status is Active')
        else : 
            #read the csv file using pandas library
            inspections_df = pd.read_csv('./dataset/Inspections.csv')
            inspections_df = inspections_df[['ACTIVITY DATE', 'SERIAL NUMBER', 'FACILITY ID', 'FACILITY NAME', 'FACILITY ZIP','PE DESCRIPTION', 'SCORE']]
            violations_df = pd.read_csv('./dataset/violations.csv')
            
            #change data type in the DataFrame
            inspections_df['SCORE'] = inspections_df['SCORE'].astype('float32')
            inspections_df['SERIAL NUMBER'] = inspections_df['SERIAL NUMBER'].astype('category')
            inspections_df['FACILITY ID'] = inspections_df['FACILITY ID'].astype('category')
            inspections_df['FACILITY NAME'] = inspections_df['FACILITY NAME'].astype('category')
            inspections_df['PE DESCRIPTION'] = inspections_df['PE DESCRIPTION'].astype('category')
            violations_df['POINTS'] = violations_df['POINTS'].astype('uint8')
            violations_df['VIOLATION  STATUS'] = violations_df['VIOLATION  STATUS'].astype('category')
            violations_df['VIOLATION CODE'] = violations_df['VIOLATION CODE'].astype('category')
            violations_df['VIOLATION DESCRIPTION'] = violations_df['VIOLATION DESCRIPTION'].astype('category')
            
            #merge the DataFrame
            inspection_vi_df = pd.merge(inspections_df, violations_df, on='SERIAL NUMBER')
            
            #save the status to the JSON File
            self.file_data['status'] = 'Import Data Files'
            self.file_data['file_name'] = './data/import_file.json'
            inspection_vi_df.to_json(self.file_data['file_name'])
            self.status_label2.configure(text=self.file_data['status'], font = ('Arial', 13), bg = 'Red')
            with open(self.file, 'w', encoding='utf-8') as make_file :
                json.dump(self.file_data, make_file, ensure_ascii=False, indent='\t')
    
    #pre-processing the DataFrame
    def prepared_file(self) :
        if(len(self.final_data) != 0) :
            msg.showinfo('Status is Active', 'Status is Active')
        else : 
            #read the status by the JSON File
            with open(self.file, 'r') as json_file :
                self.file_data = json.load(json_file)
                self.status = self.file_data['status']
                self.file_name = self.file_data['file_name']
                
            if os.path.isfile('./data/import_file.json') :
                inspection_vi_df = pd.read_json(self.file_name)
                #extract the column of the year
                inspection_vi_df['YEAR'] = inspection_vi_df['ACTIVITY DATE'].str.split('/').str[2]
                #extract the column of SEATS
                inspection_vi_df['SEATS'] = inspection_vi_df['PE DESCRIPTION'].str.extract('(\(.*\))')
                #drop the null data
                inspection_vi_df = inspection_vi_df.dropna()
                #make the final DataFrame for analyzing data.
                self.final_data = inspection_vi_df[['YEAR', 'FACILITY ID', 'FACILITY NAME', 'FACILITY ZIP', 'SEATS', 'PE DESCRIPTION', 'SCORE', 'VIOLATION CODE','VIOLATION DESCRIPTION']]
                
                #save the status to the JSON File
                self.file_data['status'] = 'Active'
                self.file_data['file_name'] = './data/prepared_file.json'
                self.final_data.to_json(self.file_data['file_name'])
                self.status_label2.configure(text=self.file_data['status'], font = ('Arial', 13), bg = 'Red')
                with open('./data/status.json', 'w', encoding='utf-8') as make_file :
                    json.dump(self.file_data, make_file, ensure_ascii=False, indent='\t')
            else :
                msg.showinfo('Need to import Data file', 'Need to import Data file')

    #show the data on the Frame    
    def show_data(self) :          
        if self.file_data['status'] == 'Active' :
            self.final_data = pd.read_json(self.file_data['file_name'])
            
            #Make the Frame
            self.f2 = Frame(self.root, height=200, width=300)
            self.f2.pack(fill=BOTH, expand=1)
            #Make and show the Table on the Frame
            self.table = Table(self.f2, dataframe=self.final_data.iloc[:, :50], read_only=True)
            self.table.show()
        elif self.final_data['status'] == 'Import Data Files' :
            msg.showinfo('Need to prepare Data file', 'Need to prepare Data file')
        elif self.final_data['status'] == 'INACTIVE' :
            msg.showinfo('Need to import Data file', 'Need to import Data file')
            
    #Save the JSON File       
    def onSave(self) :
        if(len(self.final_data) == 0) :
            msg.showinfo('No records', 'No records')
        else :
            try :
                export_file_path = filedialog.asksaveasfilename(defaultextension='.json', filetypes=[("json files", '*.json')])
                self.final_data.to_json (export_file_path)
            except Exception as e :
                msg.showerror('Error is saving file', e)

    #Open the JSON File
    def onOpen(self) :
        if(len(self.final_data) != 0) :
            msg.showinfo('Status is Active', 'Status is Active')
        else :         
            try :
                self.import_file_path = filedialog.askopenfilename(initialdir = '/Desktop')
                self.final_data = pd.read_json(self.import_file_path)
            except FileNotFoundError as e :
                msg.showerror('Error is opening file', e)
                
            #Change the status on the label
            self.status = 'ACTIVE'
            self.status_label2.configure(text=self.status, font = ('Arial', 13), bg = 'Red')

            #show the Data on the Table
            self.f2 = Frame(self.root, height=200, width=300)
            self.f2.pack(fill=BOTH, expand=1)
            self.table = Table(self.f2, dataframe=self.final_data, read_only=True)
            self.table.show()
    
    
    def onZipcode(self) :
        if(len(self.final_data) == 0) :
            msg.showinfo('No records', 'No records')
        else :
            #Make the Window
            zipWindow = Toplevel(self.root)
            zipWindow.geometry('450x250') 
            
            #Group the data which produce the mean, mode and median for the inspection score per year
            y_zip_mean = self.final_data.groupby(['YEAR', 'FACILITY ZIP'])['SCORE'].mean().reset_index().rename(columns={'SCORE' : 'Mean'})
            y_zip_mode = self.final_data.groupby(['YEAR', 'FACILITY ZIP'])['SCORE'].apply(lambda x: x.mode()).reset_index().rename(columns={'SCORE' : 'Mode'})
            y_zip_mode = y_zip_mode.drop('level_2', axis=1)
            y_zip_median = self.final_data.groupby(['YEAR', 'FACILITY ZIP'])['SCORE'].median().reset_index().rename(columns={'SCORE' : 'Median'})

            s = Frame(zipWindow, height=100, width=300)
            s.pack()
            
            #Make the label
            variable_label = Label(zipWindow, text='- ZIP CODE : ', font = ('Arial', 12))
            variable_label.place(x=50, y=20)

            #Make the optionlist for ZIPCODE
            OptionList = list(self.final_data['FACILITY ZIP'].unique())
            variable = StringVar(zipWindow)
            variable.set(OptionList[0])
            option_button = OptionMenu(zipWindow, variable, *OptionList)
            option_button.place(x=50, y=50)

            variable_label2 = Label(zipWindow, text='- Calculate : ', font = ('Arial', 12))
            variable_label2.place(x=200, y=20)

            CalList = ['MEAN', 'MODE', 'MEDIAN']
            variable2 = StringVar(zipWindow)
            variable2.set(CalList[0])
            option_button2 = OptionMenu(zipWindow, variable2, *CalList)
            option_button2.place(x=200, y=50)

            s2 = Frame(zipWindow, height=70, width=300)
            s2.pack(fill=BOTH, expand=1)
            
            #Get the initial data(MEAN) and show the data on the table
            zip_mean = y_zip_mean[y_zip_mean['FACILITY ZIP'] == variable.get()]
            table_seat = Table(s2, dataframe=zip_mean, read_only=True)
            table_seat.show()
            
            #Change the data on the table.
            def callback2(*args) :
                if variable2.get() == 'MEAN' :
                    zip_mean = y_zip_mean[y_zip_mean['FACILITY ZIP'] == variable.get()]
                    table_seat.updateModel(TableModel(zip_mean))
                    table_seat.redraw()
                elif variable2.get() == 'MODE' :
                    zip_mode = y_zip_mode[y_zip_mode['FACILITY ZIP'] == variable.get()]
                    table_seat.updateModel(TableModel(zip_mode))
                    table_seat.redraw()
                elif variable2.get() == 'MEDIAN' :
                    zip_median = y_zip_median[y_zip_median['FACILITY ZIP'] == variable.get()]
                    table_seat.updateModel(TableModel(zip_median))
                    table_seat.redraw()
            
            #Trace the callback function
            variable.trace('w', callback2)
            variable2.trace('w', callback2)
        
        
    def onSeat(self) :
        if(len(self.final_data) == 0) :
            msg.showinfo('No records', 'No records')
        else :
            #Make the Window
            seatWindow = Toplevel(self.root)
            seatWindow.geometry('450x250') 
            
            #Group the data which produce the mean, mode and median for the inspection score per year
            y_seats_mean = self.final_data.groupby(['YEAR', 'SEATS'])['SCORE'].mean().reset_index().rename(columns={'SCORE' : 'Mean'})
            y_seats_mode = self.final_data.groupby(['YEAR', 'SEATS'])['SCORE'].apply(lambda x: x.mode()).reset_index().rename(columns={'SCORE' : 'Mode'})
            y_seats_mode = y_seats_mode.drop('level_2', axis=1)
            y_seats_median = self.final_data.groupby(['YEAR', 'SEATS'])['SCORE'].median().reset_index().rename(columns={'SCORE' : 'Median'})

            s = Frame(seatWindow, height=100, width=300)
            s.pack()

            #Make the label
            variable_label = Label(seatWindow, text='- SEATS : ', font = ('Arial', 12))
            variable_label.place(x=50, y=20)
            
            #Make the optionlist for Seats
            OptionList = list(self.final_data['SEATS'].unique())
            variable = StringVar(seatWindow)
            variable.set(OptionList[0])
            option_button = OptionMenu(seatWindow, variable, *OptionList)
            option_button.place(x=50, y=50)

            variable_label2 = Label(seatWindow, text='- Calculate : ', font = ('Arial', 12))
            variable_label2.place(x=200, y=20)

            CalList = ['MEAN', 'MODE', 'MEDIAN']
            variable2 = StringVar(seatWindow)
            variable2.set(CalList[0])
            option_button2 = OptionMenu(seatWindow, variable2, *CalList)
            option_button2.place(x=200, y=50)

            s2 = Frame(seatWindow, height=70, width=300)
            s2.pack(fill=BOTH, expand=1)

            #Get the initial data(MEAN) and show the data on the table
            seats_mean = y_seats_mean[y_seats_mean['SEATS'] == variable.get()]
            table_seat = Table(s2, dataframe=seats_mean, read_only=True)
            table_seat.show()

            #Change the data on the table.
            def callback(*args) :
                if variable2.get() == 'MEAN' :
                    seats_mean = y_seats_mean[y_seats_mean['SEATS'] == variable.get()]
                    table_seat.updateModel(TableModel(seats_mean))
                    table_seat.redraw()
                elif variable2.get() == 'MODE' :
                    seats_mode = y_seats_mode[y_seats_mode['SEATS'] == variable.get()]
                    table_seat.updateModel(TableModel(seats_mode))
                    table_seat.redraw()
                elif variable2.get() == 'MEDIAN' :
                    seats_median = y_seats_median[y_seats_median['SEATS'] == variable.get()]
                    table_seat.updateModel(TableModel(seats_median))
                    table_seat.redraw()

            #Trace the callback function
            variable.trace('w', callback)
            variable2.trace('w', callback)
            
    def onNum(self) :  
        if(len(self.final_data) == 0) :
            msg.showinfo('No records', 'No records')
        else :
            #Make the Window
            numWindow = Toplevel(self.root)
            numWindow.geometry('900x600') 
            
            #Group the data which produce the number of establishments that have committed each type of violation
            violation_fac = self.final_data.groupby(['VIOLATION CODE'])['FACILITY NAME'].count().reset_index()
            violation_fac = violation_fac.set_index('VIOLATION CODE')
            
            variable_label = Label(numWindow, text='- VIOLATION CODE & NAME : ', font = ('Arial', 12))
            variable_label.place(x=50, y=20)
            
            #Make the optionlist for Violation code
            OptionList = list(self.final_data['VIOLATION CODE'].unique())
            variable = StringVar(numWindow)
            variable.set(OptionList[0])
            option_button = OptionMenu(numWindow, variable, *OptionList)
            option_button.place(x=280, y=20)
            
            #Make the label for the Violation Description
            v_name=self.final_data[['VIOLATION CODE','VIOLATION DESCRIPTION']].set_index('VIOLATION CODE').loc[variable.get()]['VIOLATION DESCRIPTION'].unique()[0]
            v_label = Label(numWindow, text=v_name, font = ('Arial', 12), bg = 'Green')
            v_label.place(x=280, y=50)
    
            f_name = []
            f_name.append(variable.get())
            
            #Make the Figure for the graph that displays the number of establishments that have committed each type of violation 
            f = plt.Figure(figsize=(12, 6))
            a = f.add_subplot(111)
            a.set(ylabel='The number of extablishments')
            #Make the graph using barplot of Seaborn library
            sns.barplot(x='VIOLATION CODE', y='FACILITY NAME', data=violation_fac.loc[f_name].reset_index(), ax=a)
            canvas = FigureCanvasTkAgg(f, numWindow)
            canvas.get_tk_widget().place(x=10, y=120)
            
            #Change the graph when call the callback function
            def callback2(*args) :
                f_name.append(variable.get())
                v_name=self.final_data[['VIOLATION CODE','VIOLATION DESCRIPTION']].set_index('VIOLATION CODE').loc[variable.get()]['VIOLATION DESCRIPTION'].unique()[0]
                v_label.configure(text=v_name, bg = 'Green')
                a.set(ylabel='The number of extablishments')
                sns.barplot(x='VIOLATION CODE', y='FACILITY NAME', data=violation_fac.loc[f_name].reset_index(), ax=a)
                canvas.draw()
            
            variable.trace('w', callback2) 
     
    def onMap(self) :  
        if(len(self.final_data) == 0) :
            msg.showinfo('No records', 'No records')
        else :
            #Pre-processing the zipcode data
            def clean_zip(x):
                return x[:5]
            self.final_data['FACILITY ZIP'] = self.final_data['FACILITY ZIP'].apply(clean_zip)
            
            #Group the data for make correlation between the number of violations committed per vendor and their zip code, 
            violation_zip = self.final_data.groupby('FACILITY ZIP').agg({'FACILITY NAME' : pd.Series.nunique}).reset_index().rename(columns = {'FACILITY NAME' : 'total'})  
            
            #Create the map by the folium library
            def create_map(table, zips, mapped_feature, add_text = ''):
                la_geo = r'./geoJSON/updated-file.json'
                m = folium.Map(location = [34.3322, -118.2437], zoom_start = 8.5)
                m.choropleth(
                    geo_data = la_geo,
                    fill_opacity = 0.7,
                    line_opacity = 0.2,
                    data = table,
                    key_on = 'feature.properties.name',
                    columns = [zips, mapped_feature],
                    fill_color = 'RdYlGn',
                    legend_name = (' ').join(mapped_feature.split('_')).title() + ' ' + add_text + ' CA'
                )
                folium.LayerControl().add_to(m)
                  
                try :
                    #Save the HTML data
                    export_file_path = filedialog.asksaveasfilename(defaultextension='.html', filetypes=[("html files", '*.html')])
                    m.save(outfile = export_file_path)
                except Exception as e :
                    msg.showerror('Error is saving file', e)
            #Craete the map
            create_map(violation_zip, 'FACILITY ZIP', 'total')
            
root = Tk()
root.title('Summative Assesment')
obj = application(root)
root.geometry('800x350') 
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas\_libs\index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'status'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-10-9c8b9b4df12f>", line 162, in show_data
    elif self.final_data['status'] == 'Import Data Files' :
  File "C:\ProgramData\Anaconda3\lib\s

## Data Analysis Program with multiprocessing method

In [3]:
#import the library for multiprocessing
import multiprocessing as mp
import prepared #prepared.py file
from multiprocessing import Process, Manager
from multiprocessing import Pool

In [4]:
class application :
    def __init__(self, root) :
        
        self.file = './data/status.json'
        
         #Store and load program status
        if os.path.isfile(self.file) :
            #Load program status from JSON File
            with open(self.file, 'r') as json_file :
                self.file_data = json.load(json_file)
                self.status = self.file_data['status']
                self.file_name = self.file_data['file_name']
                if os.path.isfile(self.file_name) :
                    self.final_data = pd.read_json(self.file_name)
                else :
                    self.final_data = pd.DataFrame()
        else :
            #Store program status from JSON File
            self.file_data = OrderedDict()
            self.status = 'INACTIVE'
            self.file_name = ''
            self.file_data['status'] = self.status
            self.file_data['file_name'] = self.file_name
            self.final_data = pd.DataFrame()
            with open(self.file, 'w', encoding='utf-8') as make_file :
                json.dump(self.file_data, make_file, ensure_ascii=False, indent='\t')
                            
        self.root = root
        
        #Craete Menubar GUI
        menubar = Menu(root)
        filemenu = Menu(menubar)
        filemenu.add_command(label='Save JSON', font = ('Arial', 12), command=self.onSave)
        filemenu.add_command(label='Open JSON', font = ('Arial', 12), command=self.onOpen)
        filemenu.add_command(label='Exit', font = ('Arial', 12), command=self.root.quit)
        
        filemenu2 = Menu(menubar)
        filemenu2.add_command(label='Vendor Seating', font = ('Arial', 12), command=self.onSeat)
        filemenu2.add_command(label='Zip Code', font = ('Arial', 12), command=self.onZipcode)
        
        filemenu3 = Menu(menubar)
        filemenu3.add_command(label='Number of Violation', font = ('Arial', 12), command=self.onNum)
        filemenu3.add_command(label='Save Violation Map', font = ('Arial', 12), command=self.onMap)
        
        #Craete Sub menubar GUI
        menubar.add_cascade(label='File', menu=filemenu, font = ('Arial', 12))
        menubar.add_cascade(label='Inspection Score', menu=filemenu2, font = ('Arial', 12))
        menubar.add_cascade(label='Visualization', menu=filemenu3, font = ('Arial', 12))
        menubar.add_cascade(label='Clear Data', font = ('Arial', 12), command=self.onClear)
        root.config(menu=menubar)
        
        #Craete Main Frame
        self.f = Frame(self.root, height=150, width=800)
        self.f.pack()
        
        #Craete Button and Label GUI
        self.generate_button = Button(self.f, text='1)Import Data Files',font = ('Arial', 13),  command=self.generate_file)
        self.clean_button = Button(self.f, text='2)Preparing Data', font = ('Arial', 13), command=self.prepared_file)
        self.show_button = Button(self.f, text='3)Show Prepared Data', font = ('Arial', 13), command=self.show_data)
        self.status_label = Label(self.f, text='- PROGRAM STATUS : ', font = ('Arial', 12))
        self.status_label2 = Label(self.f, text=self.status, font = ('Arial', 12), bg = 'Green')
        
        self.data_label = Label(self.f, text='- The prepared data set ', font = ('Arial', 12))

        #Place the widget on the frame
        self.generate_button.place(x=140, y=30)
        self.clean_button.place(x=300, y=30)
        self.show_button.place(x=450, y=30)
        self.status_label.place(x=270, y=80)
        self.status_label2.place(x=450, y=80)
        self.data_label.place(x=10, y=120)
        
        
    def onClear(self) :
        if os.path.isfile(self.file) :
            self.final_data = pd.DataFrame()
            self.file_data['status'] = 'INACTIVE'
            self.file_data['file_name'] = ''
            self.status_label2.configure(text=self.file_data['status'], font = ('Arial', 13), bg = 'Red')
            with open(self.file, 'w', encoding='utf-8') as make_file :
                json.dump(self.file_data, make_file, ensure_ascii=False, indent='\t')
            if os.path.isfile('./data/import_file.json') :
                os.remove('./data/import_file.json')
            if os.path.isfile('./data/prepared_file.json') :
                os.remove('./data/prepared_file.json')
            
            msg.showinfo('Clear Database', 'Clear Database')
        
    #import the data file(Inspections.csv, violations.csv)
    def generate_file(self) :
        if(len(self.final_data) != 0) :
            msg.showinfo('Status is Active', 'Status is Active')
        else : 
            #read the csv file using pandas library
            inspections_df = pd.read_csv('./dataset/Inspections.csv')
            inspections_df = inspections_df[['ACTIVITY DATE', 'SERIAL NUMBER', 'FACILITY ID', 'FACILITY NAME', 'FACILITY ZIP','PE DESCRIPTION', 'SCORE']]
            violations_df = pd.read_csv('./dataset/violations.csv')
            
            #change data type in the DataFrame
            inspections_df['SCORE'] = inspections_df['SCORE'].astype('float32')
            inspections_df['SERIAL NUMBER'] = inspections_df['SERIAL NUMBER'].astype('category')
            inspections_df['FACILITY ID'] = inspections_df['FACILITY ID'].astype('category')
            inspections_df['FACILITY NAME'] = inspections_df['FACILITY NAME'].astype('category')
            inspections_df['PE DESCRIPTION'] = inspections_df['PE DESCRIPTION'].astype('category')
            violations_df['POINTS'] = violations_df['POINTS'].astype('uint8')
            violations_df['VIOLATION  STATUS'] = violations_df['VIOLATION  STATUS'].astype('category')
            violations_df['VIOLATION CODE'] = violations_df['VIOLATION CODE'].astype('category')
            violations_df['VIOLATION DESCRIPTION'] = violations_df['VIOLATION DESCRIPTION'].astype('category')
            
            #merge the DataFrame
            inspection_vi_df = pd.merge(inspections_df, violations_df, on='SERIAL NUMBER')

            #save the status to the JSON File
            self.file_data['status'] = 'Import Data Files'
            self.file_data['file_name'] = './data/import_file.json'
            inspection_vi_df.to_json(self.file_data['file_name'])
            self.status_label2.configure(text=self.file_data['status'], font = ('Arial', 13), bg = 'Red')
            
            with open(self.file, 'w', encoding='utf-8') as make_file :
                json.dump(self.file_data, make_file, ensure_ascii=False, indent='\t')
   
    #pre-processing the DataFrame
    def prepared_file(self) :
        if(len(self.final_data) != 0) :
            msg.showinfo('Status is Active', 'Status is Active')
        else : 
            #read the status by the JSON File
            with open(self.file, 'r') as json_file :
                self.file_data = json.load(json_file)
                self.status = self.file_data['status']
                self.file_name = self.file_data['file_name']
                
            if os.path.isfile('./data/import_file.json') :
                inspection_vi_df = pd.read_json(self.file_name)
            
                #get the number of the cpu core count
                num_processes = mp.cpu_count()
                num_partitions = num_processes
                #split the large dataset
                df_split = np.array_split(inspection_vi_df, num_partitions)
                #create a pool object for processing input data
                pool = Pool(num_processes)
                #initiates the process creation and start processing
                inspection_vi_df = pd.concat(pool.map(prepared.prepared_data, df_split))
                #remove the pool object from memory
                pool.close()
                pool.join()
                 
                #make the final DataFrame for analyzing data.
                self.final_data = inspection_vi_df[['YEAR', 'FACILITY ID', 'FACILITY NAME', 'FACILITY ZIP', 'SEATS', 'PE DESCRIPTION', 'SCORE', 'VIOLATION CODE','VIOLATION DESCRIPTION']]

                #save the status to the JSON File
                self.file_data['status'] = 'Active'
                self.file_data['file_name'] = './data/prepared_file.json'
                self.final_data.to_json(self.file_data['file_name'])
                self.status_label2.configure(text=self.file_data['status'], font = ('Arial', 13), bg = 'Red')
                with open('./data/status.json', 'w', encoding='utf-8') as make_file :
                    json.dump(self.file_data, make_file, ensure_ascii=False, indent='\t')
            else :
                msg.showinfo('Need to import Data file', 'Need to import Data file')

        
    #show the data on the Frame    
    def show_data(self) :          
        if self.file_data['status'] == 'Active' :
            self.final_data = pd.read_json(self.file_data['file_name'])
            
            #Make the Frame
            self.f2 = Frame(self.root, height=200, width=300)
            self.f2.pack(fill=BOTH, expand=1)
            #Make and show the Table on the Frame
            self.table = Table(self.f2, dataframe=self.final_data.iloc[:, :50], read_only=True)
            self.table.show()
        elif self.final_data['status'] == 'Import Data Files' :
            msg.showinfo('Need to prepare Data file', 'Need to prepare Data file')
        elif self.final_data['status'] == 'INACTIVE' :
            msg.showinfo('Need to import Data file', 'Need to import Data file')
            
    #Save the JSON File       
    def onSave(self) :
        if(len(self.final_data) == 0) :
            msg.showinfo('No records', 'No records')
        else :
            try :
                export_file_path = filedialog.asksaveasfilename(defaultextension='.json', filetypes=[("json files", '*.json')])
                self.final_data.to_json (export_file_path)
            except Exception as e :
                msg.showerror('Error is saving file', e)

    #Open the JSON File
    def onOpen(self) :
        if(len(self.final_data) != 0) :
            msg.showinfo('Status is Active', 'Status is Active')
        else :         
            try :
                self.import_file_path = filedialog.askopenfilename(initialdir = '/Desktop')
                self.final_data = pd.read_json(self.import_file_path)
            except FileNotFoundError as e :
                msg.showerror('Error is opening file', e)
                
            #Change the status on the label
            self.status = 'ACTIVE'
            self.status_label2.configure(text=self.status, font = ('Arial', 13), bg = 'Red')

            #show the Data on the Table
            self.f2 = Frame(self.root, height=200, width=300)
            self.f2.pack(fill=BOTH, expand=1)
            self.table = Table(self.f2, dataframe=self.final_data, read_only=True)
            self.table.show()
    
    
    def onZipcode(self) :
        if(len(self.final_data) == 0) :
            msg.showinfo('No records', 'No records')
        else :
            #Make the Window
            zipWindow = Toplevel(self.root)
            zipWindow.geometry('450x250') 
            
            #Group the data which produce the mean, mode and median for the inspection score per year
            y_zip_mean = self.final_data.groupby(['YEAR', 'FACILITY ZIP'])['SCORE'].mean().reset_index().rename(columns={'SCORE' : 'Mean'})
            y_zip_mode = self.final_data.groupby(['YEAR', 'FACILITY ZIP'])['SCORE'].apply(lambda x: x.mode()).reset_index().rename(columns={'SCORE' : 'Mode'})
            y_zip_mode = y_zip_mode.drop('level_2', axis=1)
            y_zip_median = self.final_data.groupby(['YEAR', 'FACILITY ZIP'])['SCORE'].median().reset_index().rename(columns={'SCORE' : 'Median'})

            s = Frame(zipWindow, height=100, width=300)
            s.pack()
            
            #Make the label
            variable_label = Label(zipWindow, text='- ZIP CODE : ', font = ('Arial', 12))
            variable_label.place(x=50, y=20)

            #Make the optionlist for ZIPCODE
            OptionList = list(self.final_data['FACILITY ZIP'].unique())
            variable = StringVar(zipWindow)
            variable.set(OptionList[0])
            option_button = OptionMenu(zipWindow, variable, *OptionList)
            option_button.place(x=50, y=50)

            variable_label2 = Label(zipWindow, text='- Calculate : ', font = ('Arial', 12))
            variable_label2.place(x=200, y=20)

            CalList = ['MEAN', 'MODE', 'MEDIAN']
            variable2 = StringVar(zipWindow)
            variable2.set(CalList[0])
            option_button2 = OptionMenu(zipWindow, variable2, *CalList)
            option_button2.place(x=200, y=50)

            s2 = Frame(zipWindow, height=70, width=300)
            s2.pack(fill=BOTH, expand=1)
            
            #Get the initial data(MEAN) and show the data on the table
            zip_mean = y_zip_mean[y_zip_mean['FACILITY ZIP'] == variable.get()]
            table_seat = Table(s2, dataframe=zip_mean, read_only=True)
            table_seat.show()
            
            #Change the data on the table.
            def callback2(*args) :
                if variable2.get() == 'MEAN' :
                    zip_mean = y_zip_mean[y_zip_mean['FACILITY ZIP'] == variable.get()]
                    table_seat.updateModel(TableModel(zip_mean))
                    table_seat.redraw()
                elif variable2.get() == 'MODE' :
                    zip_mode = y_zip_mode[y_zip_mode['FACILITY ZIP'] == variable.get()]
                    table_seat.updateModel(TableModel(zip_mode))
                    table_seat.redraw()
                elif variable2.get() == 'MEDIAN' :
                    zip_median = y_zip_median[y_zip_median['FACILITY ZIP'] == variable.get()]
                    table_seat.updateModel(TableModel(zip_median))
                    table_seat.redraw()
            
            #Trace the callback function
            variable.trace('w', callback2)
            variable2.trace('w', callback2)
        
        
    def onSeat(self) :
        if(len(self.final_data) == 0) :
            msg.showinfo('No records', 'No records')
        else :
            #Make the Window
            seatWindow = Toplevel(self.root)
            seatWindow.geometry('450x250') 
            
            #Group the data which produce the mean, mode and median for the inspection score per year
            y_seats_mean = self.final_data.groupby(['YEAR', 'SEATS'])['SCORE'].mean().reset_index().rename(columns={'SCORE' : 'Mean'})
            y_seats_mode = self.final_data.groupby(['YEAR', 'SEATS'])['SCORE'].apply(lambda x: x.mode()).reset_index().rename(columns={'SCORE' : 'Mode'})
            y_seats_mode = y_seats_mode.drop('level_2', axis=1)
            y_seats_median = self.final_data.groupby(['YEAR', 'SEATS'])['SCORE'].median().reset_index().rename(columns={'SCORE' : 'Median'})

            s = Frame(seatWindow, height=100, width=300)
            s.pack()

            #Make the label
            variable_label = Label(seatWindow, text='- SEATS : ', font = ('Arial', 12))
            variable_label.place(x=50, y=20)
            
            #Make the optionlist for Seats
            OptionList = list(self.final_data['SEATS'].unique())
            variable = StringVar(seatWindow)
            variable.set(OptionList[0])
            option_button = OptionMenu(seatWindow, variable, *OptionList)
            option_button.place(x=50, y=50)

            variable_label2 = Label(seatWindow, text='- Calculate : ', font = ('Arial', 12))
            variable_label2.place(x=200, y=20)

            CalList = ['MEAN', 'MODE', 'MEDIAN']
            variable2 = StringVar(seatWindow)
            variable2.set(CalList[0])
            option_button2 = OptionMenu(seatWindow, variable2, *CalList)
            option_button2.place(x=200, y=50)

            s2 = Frame(seatWindow, height=70, width=300)
            s2.pack(fill=BOTH, expand=1)

            #Get the initial data(MEAN) and show the data on the table
            seats_mean = y_seats_mean[y_seats_mean['SEATS'] == variable.get()]
            table_seat = Table(s2, dataframe=seats_mean, read_only=True)
            table_seat.show()

            #Change the data on the table.
            def callback(*args) :
                if variable2.get() == 'MEAN' :
                    seats_mean = y_seats_mean[y_seats_mean['SEATS'] == variable.get()]
                    table_seat.updateModel(TableModel(seats_mean))
                    table_seat.redraw()
                elif variable2.get() == 'MODE' :
                    seats_mode = y_seats_mode[y_seats_mode['SEATS'] == variable.get()]
                    table_seat.updateModel(TableModel(seats_mode))
                    table_seat.redraw()
                elif variable2.get() == 'MEDIAN' :
                    seats_median = y_seats_median[y_seats_median['SEATS'] == variable.get()]
                    table_seat.updateModel(TableModel(seats_median))
                    table_seat.redraw()

            #Trace the callback function
            variable.trace('w', callback)
            variable2.trace('w', callback)
            
    def onNum(self) :  
        if(len(self.final_data) == 0) :
            msg.showinfo('No records', 'No records')
        else :
            #Make the Window
            numWindow = Toplevel(self.root)
            numWindow.geometry('900x600') 
            
            #Group the data which produce the number of establishments that have committed each type of violation
            violation_fac = self.final_data.groupby(['VIOLATION CODE'])['FACILITY NAME'].count().reset_index()
            violation_fac = violation_fac.set_index('VIOLATION CODE')
            
            variable_label = Label(numWindow, text='- VIOLATION CODE & NAME : ', font = ('Arial', 12))
            variable_label.place(x=50, y=20)
            
            #Make the optionlist for Violation code
            OptionList = list(self.final_data['VIOLATION CODE'].unique())
            variable = StringVar(numWindow)
            variable.set(OptionList[0])
            option_button = OptionMenu(numWindow, variable, *OptionList)
            option_button.place(x=280, y=20)
            
            #Make the label for the Violation Description
            v_name=self.final_data[['VIOLATION CODE','VIOLATION DESCRIPTION']].set_index('VIOLATION CODE').loc[variable.get()]['VIOLATION DESCRIPTION'].unique()[0]
            v_label = Label(numWindow, text=v_name, font = ('Arial', 12), bg = 'Green')
            v_label.place(x=280, y=50)
    
            f_name = []
            f_name.append(variable.get())
            
            #Make the Figure for the graph that displays the number of establishments that have committed each type of violation 
            f = plt.Figure(figsize=(12, 6))
            a = f.add_subplot(111)
            a.set(ylabel='The number of extablishments')
            #Make the graph using barplot of Seaborn library
            sns.barplot(x='VIOLATION CODE', y='FACILITY NAME', data=violation_fac.loc[f_name].reset_index(), ax=a)
            canvas = FigureCanvasTkAgg(f, numWindow)
            canvas.get_tk_widget().place(x=10, y=120)
            
            #Change the graph when call the callback function
            def callback2(*args) :
                f_name.append(variable.get())
                v_name=self.final_data[['VIOLATION CODE','VIOLATION DESCRIPTION']].set_index('VIOLATION CODE').loc[variable.get()]['VIOLATION DESCRIPTION'].unique()[0]
                v_label.configure(text=v_name, bg = 'Green')
                a.set(ylabel='The number of extablishments')
                sns.barplot(x='VIOLATION CODE', y='FACILITY NAME', data=violation_fac.loc[f_name].reset_index(), ax=a)
                canvas.draw()
            
            variable.trace('w', callback2) 
     
    def onMap(self) :  
        if(len(self.final_data) == 0) :
            msg.showinfo('No records', 'No records')
        else :
            #Pre-processing the zipcode data
            def clean_zip(x):
                return x[:5]
            self.final_data['FACILITY ZIP'] = self.final_data['FACILITY ZIP'].apply(clean_zip)
            
            #Group the data for make correlation between the number of violations committed per vendor and their zip code, 
            violation_zip = self.final_data.groupby('FACILITY ZIP').agg({'FACILITY NAME' : pd.Series.nunique}).reset_index().rename(columns = {'FACILITY NAME' : 'total'})  
            
            #Create the map by the folium library
            def create_map(table, zips, mapped_feature, add_text = ''):
                la_geo = r'./geoJSON/updated-file.json'
                m = folium.Map(location = [34.3322, -118.2437], zoom_start = 8.5)
                m.choropleth(
                    geo_data = la_geo,
                    fill_opacity = 0.7,
                    line_opacity = 0.2,
                    data = table,
                    key_on = 'feature.properties.name',
                    columns = [zips, mapped_feature],
                    fill_color = 'RdYlGn',
                    legend_name = (' ').join(mapped_feature.split('_')).title() + ' ' + add_text + ' CA'
                )
                folium.LayerControl().add_to(m)
                  
                try :
                    #Save the HTML data
                    export_file_path = filedialog.asksaveasfilename(defaultextension='.html', filetypes=[("html files", '*.html')])
                    m.save(outfile = export_file_path)
                except Exception as e :
                    msg.showerror('Error is saving file', e)
            #Craete the map
            create_map(violation_zip, 'FACILITY ZIP', 'total')
            
root = Tk()
root.title('Summative Assesment')
obj = application(root)
root.geometry('800x350') 
root.mainloop()